In [1]:
import numpy as np
import json
import sys
import pandas as pd
import os
import glob
import re
import matplotlib.pyplot as plt
import seaborn as sns
from os import listdir
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import SelectKBest


import warnings
warnings.filterwarnings("ignore")

In [2]:
from sklearn.datasets import make_hastie_10_2
from sklearn.ensemble import GradientBoostingRegressor

In [16]:
sys.path.insert(0, 'src')
from helper import *
from eda import *
from train import *                       
from etl import *

%load_ext autoreload

%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### etl (new readfilerun and gen)

In [17]:
df = readfilerun_simple('data/raw/train_r/20220116T055105_20-100-true-20-100-iperf.csv')
df

['20', '100', 'true', '20', '100']


,Time,IP1,Port1,IP2,Port2,Proto,1->2Bytes,2->1Bytes,1->2Pkts,2->1Pkts,packet_times,packet_sizes,packet_dirs,event,drop_unix,latency,loss,later_latency,later_loss,deterministic
0,1642312265,172.21.0.3,40878,172.19.0.2,5001,6,553500,18292,369,316,1642312265303;1642312265303;1642312265303;1642...,52;1500;1500;52;1500;1500;52;1500;1500;52;1500...,2;1;1;2;1;1;2;1;1;2;1;1;2;1;1;2;1;1;2;1;1;2;1;...,drop;drop;drop;drop;drop;drop;drop,1642312265.2567031;1642312265.3516712;16423122...,20,100,20,100,True
1,1642312266,172.21.0.3,40878,172.19.0.2,5001,6,235500,8432,157,155,1642312266013;1642312266013;1642312266013;1642...,52;1500;1500;52;52;1500;1500;52;1500;52;52;150...,2;1;1;2;2;1;1;2;1;2;2;1;1;2;1;2;2;1;1;2;1;2;2;...,drop;drop;drop;drop,1642312266.203085;1642312266.3627691;164231226...,20,100,20,100,True
2,1642312267,172.21.0.3,40878,172.19.0.2,5001,6,289500,9988,193,186,1642312267010;1642312267010;1642312267010;1642...,52;1500;52;52;1500;1500;52;1500;52;52;1500;150...,2;1;2;2;1;1;2;1;2;2;1;1;2;1;1;2;2;1;1;2;1;2;2;...,drop;drop;drop,1642312267.3706913;1642312267.3915396;16423122...,20,100,20,100,True
3,1642312268,172.21.0.3,40878,172.19.0.2,5001,6,405000,9228,270,173,1642312268004;1642312268004;1642312268004;1642...,52;1500;52;52;1500;1500;52;1500;52;52;1500;150...,2;1;2;2;1;1;2;1;2;2;1;1;2;2;1;1;2;1;2;2;1;1;2;...,drop;drop;drop;drop,1642312268.759712;1642312268.7804394;164231226...,20,100,20,100,True
5,1642312269,172.21.0.3,40878,172.19.0.2,5001,6,232500,3364,155,61,1642312269012;1642312269012;1642312269012;1642...,52;1500;1500;1500;1500;1500;1500;52;1500;1500;...,2;1;1;1;1;1;1;2;1;1;1;2;1;1;2;1;1;2;1;1;1;1;2;...,drop;drop;drop,1642312269.1411853;1642312269.4179938;16423122...,20,100,20,100,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301,1642312560,172.21.0.3,40878,172.19.0.2,5001,6,309000,4152,206,78,1642312560032;1642312560032;1642312560033;1642...,52;1500;52;1500;1500;1500;1500;52;1500;1500;15...,2;1;2;1;1;1;1;2;1;1;1;2;1;1;2;1;1;2;1;1;2;1;1;...,drop;drop,1642312560.413314;1642312560.8362548,20,100,20,100,True
302,1642312561,172.21.0.3,40878,172.19.0.2,5001,6,237000,3576,158,66,1642312561004;1642312561004;1642312561004;1642...,52;1500;1500;52;1500;1500;52;1500;1500;52;1500...,2;1;1;2;1;1;2;1;1;2;1;1;1;2;1;1;2;1;1;1;1;2;1;...,drop;drop;drop,1642312561.090866;1642312561.259621;1642312561...,20,100,20,100,True
303,1642312562,172.21.0.3,40878,172.19.0.2,5001,6,366000,4896,244,93,1642312562021;1642312562021;1642312562021;1642...,52;1500;1500;1500;1500;52;1500;1500;1500;1500;...,2;1;1;1;1;2;1;1;1;1;1;2;1;1;2;1;1;1;1;2;1;1;1;...,drop,1642312562.3188207,20,100,20,100,True
304,1642312563,172.21.0.3,40878,172.19.0.2,5001,6,316500,4548,211,84,1642312563039;1642312563039;1642312563039;1642...,52;1500;1500;1500;1500;52;1500;1500;1500;1500;...,2;1;1;1;1;2;1;1;1;1;2;1;1;2;1;1;1;1;2;1;1;2;1;...,drop;drop;drop,1642312563.0835528;1642312563.338909;164231256...,20,100,20,100,True


In [18]:
agg10(genfeat(df))

,total_bytes,max_bytes,proto,1->2Bytes,2->1Bytes,1->2Pkts,2->1Pkts,total_pkts,total_pkts_min,total_pkts_max,...,pkt sum,longest_seq,longest_seq_min,longest_seq_max,total_pkt_sizes,byte_ratio,mean_tdelta,max_tdelta,loss,latency
0,370373.2,7110.925503,6.0,363000.0,7373.2,242.0,134.8,376.8,216.0,685.0,...,370373.2,6.7,3.0,8.0,370373.2,1083.537122,2.778182,40.5,100.0,20.0
1,468856.8,6551.301672,6.0,462750.0,6106.8,308.5,114.9,423.4,242.0,583.0,...,468856.8,7.7,6.0,14.0,468856.8,1083.537122,2.400794,43.7,100.0,20.0
2,381860.0,2961.806926,6.0,376800.0,5060.0,251.2,94.6,345.8,262.0,441.0,...,381860.0,7.2,6.0,9.0,381860.0,1083.537122,2.844551,43.4,100.0,20.0
3,376082.8,4210.239877,6.0,371100.0,4982.8,247.4,93.2,340.6,252.0,491.0,...,376082.8,7.1,6.0,10.0,376082.8,1083.537122,2.924540,43.6,100.0,20.0
4,403559.6,3621.508194,6.0,398100.0,5459.6,265.4,102.5,367.9,230.0,495.0,...,403559.6,7.4,5.0,10.0,403559.6,1083.537122,2.745876,43.2,100.0,20.0
5,338413.6,3993.754591,6.0,333900.0,4513.6,222.6,83.9,306.5,239.0,362.0,...,338413.6,6.7,5.0,9.0,338413.6,1083.537122,3.182794,43.1,100.0,20.0
6,341859.6,3721.544267,6.0,337350.0,4509.6,224.9,84.2,309.1,230.0,450.0,...,341859.6,6.6,5.0,8.0,341859.6,1083.537122,3.212894,43.2,100.0,20.0
7,428360.8,7087.863054,6.0,422700.0,5660.8,281.8,106.4,388.2,208.0,634.0,...,428360.8,7.0,5.0,10.0,428360.8,1083.537122,2.713966,43.3,100.0,20.0
8,340528.4,4347.346143,6.0,336000.0,4528.4,224.0,84.5,308.5,211.0,480.0,...,340528.4,6.7,5.0,8.0,340528.4,1083.537122,3.335741,43.4,100.0,20.0
9,353285.6,4755.075249,6.0,348600.0,4685.6,232.4,87.6,320.0,239.0,449.0,...,353285.6,7.1,6.0,10.0,353285.6,1083.537122,3.122712,43.3,100.0,20.0


In [19]:
# df['mean_tdelta'] = df['packet_times'].str.split(';').apply(mean_diff)
# df['max_tdelta'] = df['packet_times'].str.split(';').apply(max_diff)
df['packet_times']

0      [1642312265303, 1642312265303, 1642312265303, ...
1      [1642312266013, 1642312266013, 1642312266013, ...
2      [1642312267010, 1642312267010, 1642312267010, ...
3      [1642312268004, 1642312268004, 1642312268004, ...
5      [1642312269012, 1642312269012, 1642312269012, ...
                             ...                        
301    [1642312560032, 1642312560032, 1642312560033, ...
302    [1642312561004, 1642312561004, 1642312561004, ...
303    [1642312562021, 1642312562021, 1642312562021, ...
304    [1642312563039, 1642312563039, 1642312563039, ...
305    [1642312564015, 1642312564015, 1642312564015, ...
Name: packet_times, Length: 300, dtype: object

In [32]:
def gen(cond , subset):
    unseen = ""
    if cond == 'seen': 
        print("transforming seen data")
    if cond == 'unseen': 
        print("transforming un seen data")
        unseen = "unseen"
    tempdatafiles = listdir('data/tempdata')

    data, datasubset, transformed  = [], [], []
    for j in tempdatafiles:
        loc = os.path.join(os.getcwd(), 'data', "tempdata", j)
        print(loc)
        df_cols = genfeat(pd.read_csv(loc))
        
        #data
        time_scaled = time(df_cols)
        data.append(time_scaled)
        
        #subset
        df_mid = time_scaled.iloc[60:60+subset]
        datasubset.append(df_mid)

        #transformed
        f_df = agg10(df_cols)
        transformed.append(f_df)
        
    # makes paths
    path = os.path.join(os.getcwd() , "outputs", "gen_temp")
    path2 = os.path.join(os.getcwd() , "outputs")
    
    list_to_csv(data, os.path.join(path2, unseen + "combined_all.csv"))
    print('combined_finished', sep=' ')
    list_to_csv(datasubset, os.path.join(path2, unseen + "combined_subset_6068.csv"))
    print('combined_all_finished', sep=' ')
    list_to_csv(transformed, os.path.join(path2, unseen +  "combined_transform.csv"))
    print('transformed_finished', sep=' ')
        
    return None
def list_to_csv(lst, filepth):
    '''takes list of pandas dataframes with similar column structure and outputs them to a single folder'''
    lst[0].to_csv(filepth, index=False)
    for i in range(1, len(lst)):
        lst[i].to_csv(filepth, index=False, header=False, mode='a')
    return


In [33]:
def gen_helper(ct, file_lst, subset, unseen):
    """ """
    data, datasubset, transformed  = [], [], []
    for j in file_lst:
        loc = os.path.join(os.getcwd(), "tempdata", j)
        print(loc)
        df_cols = genfeat(pd.read_csv(loc))
        
        #data
        time_scaled = time(df_cols)
        data.append(time_scaled)
        
        #subset
        df_mid = time_scaled.iloc[60:60+subset]
        datasubset.append(df_mid)

        #transformed
        f_df = agg10(df_cols)
        transformed.append(f_df)

    path = os.path.join(os.getcwd() , "outputs", "gen_temp")

    temp_data = pd.concat(data , ignore_index=True)#.reset_index(drop = True)
    temp_data.to_csv(os.path.join(path, unseen + "temp_all_" + str(ct) + ".csv"), index = False)
    
    temp_subset = pd.concat(datasubset , ignore_index=True)
    temp_subset.to_csv(os.path.join(path, unseen +  "temp_subset_6068_" + str(ct) + ".csv"), index = False)
    
    temp_t = pd.concat(transformed, ignore_index=True)  
    temp_t.to_csv(os.path.join(path, unseen +  "temp_transform_" + str(ct) + ".csv"), index = False)

#### eda

In [34]:
def main_eda(cond, lst, filen1, filen2, filen3):
    unseen = ''
    if cond =='unseen': 
        unseen = 'unseen'
    
    fpath1 = os.path.join(os.getcwd() , "outputs", unseen + filen1)
    df_1 = pd.read_csv(fpath1)
    fpath2 = os.path.join(os.getcwd() , "outputs", unseen + filen2)
    df_2 = pd.read_csv(fpath2)
    fpath3 = os.path.join(os.getcwd() , "outputs", unseen + filen3)
    df_3 = pd.read_csv(fpath3)
    
    plottogether(cond, lst, df_1, filen1.strip(".csv")) # trends over subset
    plottogether(cond, lst, df_3, filen3.strip(".csv")) # trends over entire data
    plotloss(cond, df_2)

    plot_correlation_matrix(cond, df_2) # correlation matrix
    plotlongest(df_3, cond)
    # below makes rest of visualizations
    plotbytes(df_3)
    #plot_detailed_bytes()

In [35]:
# def plot_detailed_bytes(df):
#     ax = plt.figure(figsize=(18,8))
#     df['1->2Bytes'].plot(title='1->2Bytes/s Rate')
#     plt.axvline(x=180, color='r')
#     for i in df[df['event'] == 'drop'].index:
#         plt.axvline(x=i, color='y', alpha=.45)
#     custom_lines = [Line2D([0], [0], color='b', lw=2),
#         Line2D([0], [0], color='y', lw=2, alpha=0.45),
#         Line2D([0], [0], color='r', lw=2)]
#     plt.legend(custom_lines, ['Bytes per Second', 'Packet drop', '180s mark'], loc='upper right', framealpha=1);
def plot_detailed_bytes(df, col='1->2Bytes', rollsec=10):
    rollcolor = '#6c2b6d'
    detailcolor = '#e98d6b'
    
    ax = plt.figure(figsize=(18,8))
    df[col].plot(title=f'{col}/s Rate', color=detailcolor)
    df[col].rolling(rollsec).mean().bfill().plot(color=rollcolor)
    plt.axvline(x=180, color='r')
    for i in df[df['event'] == 'drop'].index:
        plt.axvline(x=i, color='y', alpha=.45)
    custom_lines = [Line2D([0], [0], color=detailcolor, lw=2),
        Line2D([0], [0], color=rollcolor, lw=2),
        Line2D([0], [0], color='y', lw=2, alpha=0.45),
        Line2D([0], [0], color='r', lw=2)]
    plt.legend(custom_lines, 
               [f'{col} per Second', f'{col} per Second ({rollsec}s rolling avg)', 'Packet drop', '180s Mark'], 
               loc='upper right', framealpha=1);

In [36]:
# overlaying model predictions and plotting detailed bytes

#### train

In [37]:
def test_feat(cond, df, cols, p, df_u): 
    unseen = ''
    if cond =='unseen': 
        unseen = 'unseen'
    # col is feauture comb
    # p is for loss or latency   1: loss  # 2 : latency
    X = df[cols]
    X2 = df_u[cols]

    if p == 1: 
        y = df.loss
        y2 = df_u.loss
    if p == 2: 
        y = df.latency
        y2 = df_u.latency
        
    # randomly split into train and test sets, test set is 80% of data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
    X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.2, random_state=1)

    if unseen == 'unseen': 
        X_test = X2
        y_test = y2
    
    clf = DecisionTreeRegressor()
    clf = clf.fit(X_train,y_train)
    y_pred = clf.predict(X_test)
    acc1 = mean_squared_error(y_test, y_pred)
    
    clf2 = RandomForestRegressor(n_estimators=200)
    clf2 = clf2.fit(X_train,y_train)
    y_pred2 = clf2.predict(X_test)
    acc2= mean_squared_error(y_test, y_pred2)
    #print("Random Forest Accuracy:", acc2, '\n')
    
    clf3 = ExtraTreesRegressor(n_estimators=200)
    clf3 = clf3.fit(X_train,y_train)
    y_pred3 = clf3.predict(X_test)
    acc3= mean_squared_error(y_test, y_pred3)
    #print("Extra Trees Accuracy:", acc3, '\n')
    
    pca = PCA() 
    X_transformed = pca.fit_transform(X_train) 
    cl = DecisionTreeRegressor() 
    cl.fit(X_transformed, y_train)
    newdata_transformed = pca.transform(X_test)
    y_pred4 = cl.predict(newdata_transformed)
    acc4 = mean_squared_error(y_test, y_pred4)
    #print("PCA Accuracy:", acc4, '\n')
    
    clf_gbc = GradientBoostingRegressor(random_state=0)
    clf_gbc.fit(X_train, y_train)
    y_pred5 = clf_gbc.predict(X_test)
    acc5 = mean_squared_error(y_test, y_pred5)
    return [acc1, acc2, acc3, acc4, acc5]

# def train_feat():
    


def test_mse(cond, all_comb1, all_comb2):
    unseen = ''
    if cond =='unseen': 
        unseen = 'unseen'
    filedir_unseen = os.path.join(os.getcwd(), "outputs", unseen + "combined_transform.csv")
    df_unseen = pd.read_csv(filedir_unseen)
    filedir = os.path.join(os.getcwd(), "outputs", "combined_transform.csv")
    df = pd.read_csv(filedir)
    
    all_comb1 = pd.Series(all_comb1).apply(lambda x: list(x))
    all_comb2 = pd.Series(all_comb2).apply(lambda x: list(x))
    dt = []
    rf = []
    et = []
    pca = []
    gbc = []
    for i in all_comb1:
        acc_loss = test_feat(cond, df, i, 1, df_unseen)
        dt.append(acc_loss[0])
        rf.append(acc_loss[1])
        et.append(acc_loss[2])
        pca.append(acc_loss[3])
        gbc.append(acc_loss[4])

    dt2 = []
    rf2 = []
    et2 = []
    pca2 = []
    gbc2 = []
    for i in all_comb2:
        # 1 = loss
        # 2 = latency
        acc_latency = test_feat(cond, df, i, 2, df_unseen)
        #print(accs)
        dt2.append(acc_latency[0])
        rf2.append(acc_latency[1])
        et2.append(acc_latency[2])
        pca2.append(acc_latency[3])
        gbc2.append(acc_latency[4])
    
    dict1 = pd.DataFrame({'feat': all_comb1, 'dt': dt, 'rf': rf, 'et': et, 'pca': pca, 'gbc': gbc})
    dict2 = pd.DataFrame({'feat2': all_comb2, 'dt2': dt2, 'rf2': rf2, 'et2': et2, 'pca2': pca2, 'gbc2': gbc2})
    
    #feat_df = pd.concat([dict1, dict2], axis=1).drop(['feat2'], axis=1)
    path = os.path.join(os.getcwd() , "outputs")
    dict1.to_csv(os.path.join(path, unseen + "feat_df1.csv"), index = False)
    dict2.to_csv(os.path.join(path, unseen + "feat_df2.csv"), index = False)
    
    # return feat_df

def best_performance(cond):
    unseen = ''
    if cond == 'unseen': 
        unseen = 'unseen'
    #print("finding best loss performance")
    filedir1 = os.path.join(os.getcwd(), "outputs", unseen + "feat_df1.csv")
    df1 = pd.read_csv(filedir1)
    print( "\n")
    print("Loss Performance sorted from lowest to highest", "\n")
    print(df1.sort_values(by=['dt', 'rf', 'et', 'pca'], ascending = True)[:5], "\n")
    #print("Loss Performance sorted from highest to lowest")
    #print(df1.sort_values(by=['dt', 'rf', 'et', 'pca'], ascending = False)[:5])
    

    #print("finding best latency performance")
    filedir2 = os.path.join(os.getcwd(), "outputs", unseen + "feat_df2.csv")
    df2 = pd.read_csv(filedir2)
    print( "\n")
    print("Latency Performance sorted from lowest to highest", "\n")
    print(df2.sort_values(by=['dt2', 'rf2', 'et2', 'pca2'], ascending = True)[:5], "\n")
    #print("Latency Performance sorted from highest to lowest")
    #print(df2.sort_values(by=['dt2', 'rf2', 'et2', 'pca2'], ascending = False)[:5])
    
    #combined.to_csv("combined_latency.csv")
    # sorted.to_csv(os.path.join(path, "sorted_df.csv"), index = False)


#### main

In [38]:
def main(targets):

    transform_config = json.load(open('config/transform.json'))
    columns = json.load(open('config/columns.json'))
    eda_config = json.load(open('config/eda.json'))
    all_config = json.load(open("config/all.json"))

    test_unseen = 'unseen'
    test_seen = 'seen'
    
    cond1 = True
    cond2 = False

    if 'data' in targets:
        """generating feat from unseen and seen data"""
        readfilerun('data2', 'tempdata') 
        gen(test_seen, **transform_config)
#         gen(test_unseen, **transform_config)

    if 'eda' in targets:  
#         readfilerun('data2', 'tempdata') 
#         gen(test_seen, **transform_config)
        main_eda(test_seen, **eda_config)
        print("EDA saved to outputs/eda/ folder")

    if 'train' in targets:
        "trains tests in this target"
#         readfilerun('data2', 'tempdata') 
#         gen(test_seen, **transform_config)
                
        comb1 = getAllCombinations(1)
        comb2 = getAllCombinations(2)
        
        print("Testing on seen data: ")
        test_mse(test_seen, comb1, comb2)
        best_performance(test_seen)
                        
    if "inference" in targets: 
        print('tba')
#         readfilerun('data2', 'tempdata') 
#         gen(test_seen, **transform_config)
#         gen(test_unseen, **transform_config)
        
#         comb1 = getAllCombinations(1)
#         comb2 = getAllCombinations(2)
        
#         print("Testing on unseen data: ")
#         test_mse(test_unseen, comb1, comb2)
#         best_performance(test_unseen)
            
    if "test" in targets: 
        """ runs all targets on sample data"""
        print('tba')
#         readfilerun('data2', 'tempdata') 
#         gen(test_seen, **transform_config)
#         gen(test_unseen, **transform_config)
        
#         main_eda(test_seen, **eda_config)
#         print("EDA saved to outputs/eda/ folder")
        
#         comb1 = getAllCombinations(1)
#         comb2 = getAllCombinations(2)
        
#         print("Testing on seen data: ")
#         test_mse(test_seen, comb1, comb2)
#         best_performance(test_seen)
        
#         print("Testing on unseen data: ")
#         test_mse(test_unseen, comb1, comb2)
#         best_performance(test_unseen)
        
    if 'all' in targets: 
        print('tba')
        
# if __name__ == '__main__':

#     targets = sys.argv[1:]
#     main(targets)

In [41]:
targets = ['data', 'eda', 'train', 'inference', 'test', 'all']
main(targets[0])

transforming seen data


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'data/tempdata'

In [40]:
main(targets[2])

all combinations generated
all combinations generated
Testing on seen data: 


FileNotFoundError: [Errno 2] No such file or directory: 'c:\\Users\\Staro\\Documents\\GitHub\\Q2\\outputs\\combined_transform.csv'

In [ ]:

def plot_detailed_bytes(df, col='1->2Bytes', rollsec=10):
    rollcolor = '#6c2b6d'
    detailcolor = '#e98d6b'
    
    ax = plt.figure(figsize=(18,8))
    df[col].plot(title=f'{col}/s Rate', color=detailcolor)
    df[col].rolling(rollsec).mean().bfill().plot(color=rollcolor)
    plt.axvline(x=180, color='r')
    for i in df[df['event'] == 'drop'].index:
        plt.axvline(x=i, color='y', alpha=.45)
    custom_lines = [Line2D([0], [0], color=detailcolor, lw=2),
        Line2D([0], [0], color=rollcolor, lw=2),
        Line2D([0], [0], color='y', lw=2, alpha=0.45),
        Line2D([0], [0], color='r', lw=2)]
    plt.legend(custom_lines, 
               [f'{col} per Second', f'{col} per Second ({rollsec}s rolling avg)', 'Packet drop', '180s Mark'], 
               loc='upper right', framealpha=1);

df.column for predicted value
df.column for actual label

In [21]:
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import make_scorer

df = pd.read_csv('outputs\\combined_transform.csv')
df


,total_bytes,max_bytes,proto,1->2Bytes,2->1Bytes,1->2Pkts,2->1Pkts,total_pkts,number_ms,pkt_ratio,time_spread,pkt sum,longest_seq,total_pkt_sizes,byte_ratio,loss,latency
0,147546.4,3977.820375,6.0,143716.4,3830.0,96.0,70.9,166.9,36.3,928.661778,905.6,147546.4,5.0,147546.4,1066.509474,100.0,100.0
1,82518.8,2954.589921,6.0,81300.0,1218.8,54.2,22.5,76.7,17.6,1073.823761,901.3,82518.8,5.1,82518.8,1066.509474,100.0,100.0
2,72773.6,2132.137165,6.0,71550.0,1223.6,47.7,22.3,70.0,20.0,1037.201125,920.1,72773.6,4.6,72773.6,1066.509474,100.0,100.0
3,77979.2,2056.799196,6.0,76800.0,1179.2,51.2,22.1,73.3,20.4,1063.096983,916.3,77979.2,4.5,77979.2,1066.509474,100.0,100.0
4,148404.0,2450.035501,6.0,146400.0,2004.0,97.6,37.2,134.8,27.5,1099.955806,960.5,148404.0,6.4,148404.0,1066.509474,100.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7702,1181395.2,4610.190760,6.0,1167000.0,14395.2,778.0,270.9,1048.9,125.5,1128.508362,911.9,1181395.2,7.6,1181395.2,1105.379896,9000.0,500.0
7703,1352562.4,5613.342138,6.0,1335600.0,16962.4,890.4,326.2,1216.6,140.1,1110.808290,953.7,1352562.4,7.5,1352562.4,1105.379896,9000.0,500.0
7704,2152011.2,113913.653088,6.0,2121928.8,30082.4,1414.8,536.3,1951.1,211.4,1098.366122,664.8,2152011.2,220.4,2152011.2,1105.379896,9000.0,500.0
7705,1420124.8,134999.098851,6.0,1401492.0,18632.8,934.6,324.6,1259.2,151.1,1125.585269,668.7,1420124.8,174.8,1420124.8,1105.379896,9000.0,500.0


In [24]:
indexcol = ['total_bytes','max_bytes','2->1Bytes','2->1Pkts','total_pkts', 'total_pkts_min', 'total_pkts_max', 'number_ms', 'pkt_ratio','time_spread', 'time_spread_min','time_spread_max','pkt sum','longest_seq', 'longest_seq_min', 'longest_seq_max','total_pkt_sizes','byte_ratio', 'mean_tdelta', 'max_tdelta']
len(indexcol)

20

['total_bytes',
 'max_bytes',
 '2->1Bytes',
 '2->1Pkts',
 'total_pkts',
 'number_ms',
 'pkt_ratio',
 'time_spread',
 'pkt sum',
 'longest_seq',
 'total_pkt_sizes',
 'byte_ratio']

In [45]:
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import make_scorer

label_col = 'latency'

df = pd.read_csv('outputs\\combined_transform.csv')

indexcol = ['total_bytes','max_bytes','2->1Bytes','2->1Pkts','total_pkts', 'total_pkts_min', 'total_pkts_max', 'number_ms', 'pkt_ratio','time_spread', 'time_spread_min','time_spread_max','pkt sum','longest_seq', 'longest_seq_min', 'longest_seq_max','total_pkt_sizes','byte_ratio', 'mean_tdelta', 'max_tdelta']
len(indexcol)

X_train, X_test, y_train, y_test = train_test_split(
    df[[x for x in indexcol if x in df.columns]], df[label_col])
# print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
etree = ExtraTreesRegressor(n_estimators=400, n_jobs=4)
etreeft = etree.fit(X_train,y_train)

y_pred3 = etree.predict(X_test)
acc3= mean_squared_error(y_test, y_pred3)

print(f'mse: {acc3}, r2: {etree.score(X_test, y_test)}')
feat_imp = pd.Series(index=[x for x in indexcol if x in df.columns], 
          data=etree.feature_importances_).sort_values(ascending=False)
feat_imp

(5780, 12) (1927, 12) (5780,) (1927,)
mse: 1709.884903347172, r2: 0.9318839253945324


byte_ratio         0.553971
time_spread        0.102298
number_ms          0.075867
pkt_ratio          0.072764
max_bytes          0.032258
2->1Pkts           0.025018
total_pkt_sizes    0.024187
2->1Bytes          0.023896
total_bytes        0.023580
pkt sum            0.023410
total_pkts         0.023066
longest_seq        0.019684
dtype: float64

In [32]:
acc3

1888.4178210949665

In [44]:
print(f'mse: {acc3}, r2: {etree.score(X_test, y_test)}')
feat_imp = pd.Series(index=[x for x in indexcol if x in df.columns], 
          data=etree.feature_importances_).sort_values(ascending=False)
feat_imp

mse: 1888.4178210949665, r2: 0.9239026238224772


byte_ratio         0.570085
time_spread        0.099886
pkt_ratio          0.069825
number_ms          0.069792
max_bytes          0.028858
pkt sum            0.024635
total_pkts         0.023934
total_pkt_sizes    0.023690
2->1Bytes          0.022993
2->1Pkts           0.022918
total_bytes        0.022051
longest_seq        0.021332
dtype: float64

In [43]:
pd.Series(index=[x for x in indexcol if x in df.columns], 
          data=etree.feature_importances_).sort_values(ascending=False)


byte_ratio         0.570085
time_spread        0.099886
pkt_ratio          0.069825
number_ms          0.069792
max_bytes          0.028858
pkt sum            0.024635
total_pkts         0.023934
total_pkt_sizes    0.023690
2->1Bytes          0.022993
2->1Pkts           0.022918
total_bytes        0.022051
longest_seq        0.021332
dtype: float64